### 0.대전광역시_대전교통정보 open API
#### - 변수설명

1. congestion : 혼잡코드   
  - 0:정보없음 1:소통원활 2:지체 3:정체
2. linkCount: 링크 개수
3. linkID: 링크식별번호
4. linkLength: 링크길이
5. linkSqc:	링크정렬순번
6. speed: 통행속도(Km/h)
7. roadName: 도로명칭	
8. startNodeId: 시작노드ID
9. 	startNodeName: 시작노드명
10. endNodeID: 끝노드ID
11. endNodeName: 끝노드명	
12. travelT: 통행시간(초)	
8. udType: 방향정보   
  - 0:정보없음, 1:상행, 2:하행

### 1. 필요 라이브러리

In [ ]:
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote

### 2. Open API 데이터 불러오기

In [2]:
# 1. URL 파라미터 분리하기.
# Service URL
xmlUrl = 'http://openapitraffic.daejeon.go.kr/traffic/rest/getTrafficInfoAll.do'
My_API_Key = unquote('3kJRZ3NtzNR4cl5IjM9Fk9B4TXbkgBLFdZEXRBQopcmTFaqtmZq%2FBdgYdh9Y%2F%2FrJUIqmNGl58UUPhbHG0HD2JA%3D%3D')    


In [ ]:
# My_API_Key = unquote('Agq7hySmyMi1FFU9kYibP%2BEnxYepQ%2FB6Dn%2Bw9lsYKVSCDjTwIdvpjmuhJrtyQrhipg3F3a4jbSq%2FLxHi%2FdUIoQ%3D%3D')    # 사용자 인증키
queryParams = '?' + urlencode(    # get 방식으로 쿼리를 분리하기 위해 '?'를 넣은 것이다. 메타코드 아님.
    {
        quote_plus('ServiceKey') : My_API_Key,    # 필수 항목 1 : ServiceKey
        quote_plus('pageNo') : '0',          # 필수 항목 2 : pageNo
        quote_plus('numOfRows') : '10'         # 픽수 항목 3 : 	numOfRows
     }
)

response = requests.get(xmlUrl + queryParams).text.encode('utf-8')
xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')
# xmlobj    # 디버깅용.

In [ ]:
rows = xmlobj.findAll('TRAFFIC')

In [22]:
#한개 행 모든컬럼값 리스티에 담기
columns = rows[0].find_all()
columns

[<congestion>1</congestion>,
 <linkCount>14444</linkCount>,
 <linkID>1830000100</linkID>,
 <linkLength>145.293874194</linkLength>,
 <linkSqc>1</linkSqc>,
 <speed>30</speed>,
 <roadName>일반국도17호선</roadName>,
 <startNodeId>1830000100</startNodeId>,
 <startNodeName>추부터널</startNodeName>,
 <endNodeID>2920011700</endNodeID>,
 <endNodeName>추부터널</endNodeName>,
 <travelT>5</travelT>,
 <udType>2</udType>]

In [23]:
rowList = []
nameList = []
columnList = []

columnsLen = len(columns)
for j in range(0, columnsLen):
    eachColumn = columns[j].text
    columnList.append(eachColumn)
rowList.append(columnList)

In [18]:
# 모든 행과 열의 값을 모아 매트릭스로 만들어보자.
rowList = []
nameList = []
columnList = []

rowsLen = len(rows)
for i in range(0, rowsLen):
    columns = rows[i].find_all()
    
    columnsLen = len(columns)
    for j in range(0, columnsLen):
        # 첫 번째 행 데이터 값 수집 시에만 컬럼 값을 저장한다. (어차피 rows[0], rows[1], ... 모두 컬럼헤더는 동일한 값을 가지기 때문에 매번 반복할 필요가 없다.)
        if i == 0:
            nameList.append(columns[j].name)        # 컬럼값은 모든 행의 값을 저장해야한다.    
        eachColumn = columns[j].text
        columnList.append(eachColumn)
    rowList.append(columnList)
    columnList = []    # 다음 row의 값을 넣기 위해 비워준다. (매우 중요!!)

result = pd.DataFrame(rowList, columns=nameList)

In [19]:
result

,congestion,linkCount,linkID,linkLength,linkSqc,speed,roadName,startNodeId,startNodeName,endNodeID,endNodeName,travelT,udType
0,1,14444,1830000100,145.293874194,1,30,일반국도17호선,1830000100,추부터널,2920011700,추부터널,5,2
1,1,14444,1830000200,145.940446066,2,30,태봉길,2920011700,추부터널,1830000100,추부터널,5,1
2,1,14444,1830000301,210.42007237,3,30,태봉길,1830000100,추부터널,1830000101,하소동,7,1
3,2,14444,1830000302,10982.3911923,4,16,일반국도17호선,1830000101,하소동,1830001805,구도동,686,1
4,1,14444,1830000303,382.712429313,5,30,산내길,1830001805,구도동,1830001804,석천들주공아파트,13,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,1,14444,1830000401,211.001711258,6,30,태봉길,1830000101,하소동,1830000100,추부터널,7,2
61,1,14444,1830000403,389.776300282,7,30,산내길,1830001804,석천들주공아파트,1830001805,구도동,13,2
62,1,14444,1830001904,475.395201721,8,30,산내길,1830001804,석천들주공아파트,1830001803,낭월교네거리,16,1
63,2,14444,1830001905,460.149597301,9,15.24,산내길,1830001803,낭월교네거리,1830001802,산내주공아파트,30,1


In [100]:
result.to_excel('대전교통정보.xlsx',index=False)